In [69]:
import pandas as pd
import numpy as np

from warnings import filterwarnings
filterwarnings('ignore')

In [70]:
original_file = './data/netflix_user_data_unclean.csv'
df = pd.read_csv(original_file)

# 무의미한 컬럼 제거
df.drop(['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18'], axis=1, inplace=True)

In [71]:
# 숫자형과 문자형 컬럼 분리
numerical_cols = [
    'Subscription Length (Months)', 'Customer Satisfaction Score (1-10)', 'Daily Watch Time (Hours)', 'Engagement Rate (1-10)',
    'Support Queries Logged', 'Age', 'Monthly Income ($)', 'Promotional Offers Used', 'Number of Profiles Created'
]

categorical_cols = [
    'Customer ID', 'Device Used Most Often', 'Genre Preference', 'Region', 'Payment History (On-Time/Delayed)', 'Subscription Plan', 'Churn status'
]

In [72]:
# 숫자형으로 변환하고 변환할 수 없을 경우 NaN으로 취급
for col in numerical_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

### 피처 엔지니어링

In [73]:
# Churn status 컬럼명 변경
df.rename(columns={'Churn status': 'Churn'}, inplace=True)

In [74]:
df.drop(index=df[df['Churn'].isnull()].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [75]:
# 10살 간격으로 연령대 컬럼 생성
min_age = df['Age'].min()
max_age = df['Age'].max()

bin_from = int(np.floor(min_age / 10) * 10)
bin_to = int(np.ceil(max_age / 10) * 10)

bins = list(range(bin_from, bin_to + 10, 10))
bins.append(99)
labels = [i for i in bins[:-1]]

df['Age_group'] = pd.cut(x=df['Age'], bins=bins, labels=labels, right=False)
df.drop('Age', axis=1, inplace=True)

In [76]:
df_price = pd.read_csv('./data/country_sub_fee_raw.csv')
df_price

,country_code,country1,country2,region,currency,price_basic,price_mobile,price_premium,price_standard,price_standard_with_ads,price_usd_basic,price_usd_mobile,price_usd_premium,price_usd_standard,price_usd_standard_with_ads,region.1
0,AD,Andorra,Andorra,Europe,EUR,8.99,NaN,19.99,13.99,NaN,10.10,NaN,22.46,15.72,NaN,Europe
1,AE,United Arab Emirates,United Arab Emirates,Asia,AED,35.00,NaN,71.00,49.00,NaN,9.53,NaN,19.34,13.35,NaN,Asia
2,AF,Afghanistan,Afghanistan,Asia,USD,3.99,NaN,9.99,7.99,NaN,3.99,NaN,9.99,7.99,NaN,Asia
3,AG,Antigua & Barbuda,Antigua & Barbuda,North America,USD,10.99,NaN,19.99,15.49,NaN,10.99,NaN,19.99,15.49,NaN,North America
4,AI,Anguilla,Anguilla,North America,USD,10.99,NaN,19.99,15.49,NaN,10.99,NaN,19.99,15.49,NaN,North America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,YE,Yemen,Yemen,Asia,USD,3.99,NaN,9.99,7.99,NaN,3.99,NaN,9.99,7.99,NaN,Asia
240,YT,Mayotte,Mayotte,Africa,EUR,11.99,NaN,21.99,14.99,NaN,13.47,NaN,24.71,16.84,NaN,Africa
241,ZA,South Africa,South Africa,Africa,ZAR,99.00,59.00,229.00,179.00,NaN,5.48,3.26,12.67,9.90,NaN,Africa
242,ZM,Zambia,Zambia,Africa,USD,3.99,2.99,9.99,7.99,NaN,3.99,2.99,9.99,7.99,NaN,Africa


In [77]:
df_price.rename(columns={'region': 'Region'}, inplace=True)

In [78]:
# Region별 평균 요금(결합용 데이터프레임 따로 생성)
price_cols = ['price_usd_basic', 'price_usd_standard', 'price_usd_premium']
df_region_price = df_price.groupby('Region')[price_cols].mean().reset_index()

df_region_price.rename(
    columns={
        'price_usd_basic': 'avg_region_price_basic',
        'price_usd_standard': 'avg_region_price_standard',
        'price_usd_premium': 'avg_region_price_premium'
    }, inplace=True
)

# 사용자 데이터와 지역별 평균 요금 데이터 결합
df_merged = pd.merge(df, df_region_price, on='Region', how='left')

In [79]:
df_region_price

,Region,avg_region_price_basic,avg_region_price_standard,avg_region_price_premium
0,Africa,4.300000,8.117368,10.299298
1,Antarctica,10.990000,16.160000,22.334000
2,Asia,6.500000,9.955400,12.836000
3,Europe,9.544634,14.418627,19.483529
4,North America,8.475000,12.473659,16.641220
5,Oceania,7.154211,12.846154,17.038462
6,South America,6.822308,9.744286,13.218571


In [ ]:
# 전체 데이터에 대한 결측치 처리
# 모두 제거
for col in df_merged.select_dtypes(include=np.number).columns:
    df_merged.drop(df_merged[df_merged[col].isnull()].index, inplace=True)
    df_merged.reset_index(drop=True, inplace=True)

for col in df_merged.select_dtypes(include='object').columns:
    df_merged.drop(df_merged[df_merged[col].isnull()].index, inplace=True)
    df_merged.reset_index(drop=True, inplace=True)

In [81]:
# 실제 구독 요금 계산 컬럼 생성
conditions = [
    (df_merged['Subscription Plan'] == 'Basic'),
    (df_merged['Subscription Plan'] == 'Standard'),
    (df_merged['Subscription Plan'] == 'Premium')
]
choices = [
    df_merged['avg_region_price_basic'],
    df_merged['avg_region_price_standard'],
    df_merged['avg_region_price_premium']
]

df_merged['User_Subscription_Price'] = np.select(conditions, choices, default=0)

In [82]:
# 새로운 복합 컬럼 생성
epsilon = 1e-6 # 0으로 나누는 것을 방지하기 위한 작은 값

# 1. 소득 대비 요금
df_merged['Price_Burden_Ratio'] = df_merged['User_Subscription_Price'] / (df_merged['Monthly Income ($)'] + epsilon)

# 2. 요금별 시청시간(1달러당 얼마나 보는지)
df_merged['Watch_Time_per_Dollar'] = df_merged['Daily Watch Time (Hours)'] / (df_merged['User_Subscription_Price'] + epsilon)

# 3. 요금별 만족도
df_merged['Satisfaction_per_Dollar'] = df_merged['Customer Satisfaction Score (1-10)'] / (df_merged['User_Subscription_Price'] + epsilon)

# 4. 월 평균 문의 수
df_merged['Queries_per_Month'] = df_merged['Support Queries Logged'] / (df_merged['Subscription Length (Months)'] + epsilon)

# 무한대 값 처리
df_merged.replace([np.inf, -np.inf], 0, inplace=True)

In [83]:
df_merged.to_csv('./data/netflix_feature_engineered.csv', index=False)

In [85]:
df_region_price.to_csv('./data/region_price.csv', index=False)